In [1]:
!pip install requests
!pip install selenium

In [15]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from urllib.parse import urlparse
import webbrowser
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import config
import json

In [3]:
client_id = config.client_id
client_secret = config.client_secret

In [4]:
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_did_expire = True
    client_id = None
    client_secret = None
    did_perform_grant_flow_auth = False
    authorize_url = "https://accounts.spotify.com/authorize"
    redirected_url = None
    scope = None
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("client_secret or client_id is None")
        client_credentianls = f"{client_id}:{client_secret}"
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_token_headers(self):
        return {
            "Authorization": f"Basic {client_credentials_b64.decode()}"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        request = requests.post(token_url, data=token_data, headers=token_headers)
        if request.status_code in range(200,299):
            token_response_data = request.json()
            current_time = datetime.datetime.now()
            access_token = token_response_data['access_token']
            expires_in_seconds = token_response_data['expires_in']
            expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
            self.access_token = access_token
            self.access_token_expires = expires
            self.access_token_did_expire = expires < current_time
            return True
        return False
    
    def perform_auth_grant_flow(self):
        client_id = self.client_id
        authorize_url = self.authorize_url
        redirect_uri = "http://localhost:8888/callback"
        authorize_url = self.authorize_url
        scope = self.scope
        if scope is None:
            raise Exception("lack of scope, set permissions scope")
        
        parameters = urlencode({"client_id": client_id, "redirect_uri": redirect_uri, "response_type": "token", 
                                "scope": scope})
        request = requests.get(f"{authorize_url}?{parameters}")
        if request.status_code in range(200,299):
            #webbrowser.open(f"{authorize_url}?{parameters}")
            driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
            driver.get(f"{authorize_url}?{parameters}")
            try:
                WebDriverWait(driver, 60).until(EC.url_contains("access"))
            finally:
                url = driver.current_url
                driver.quit();
                return url
        else:
            raise Exception("failed to request")
    
    def set_redirected_url(self, url):
        self.redirected_url = url
        
    def get_redirected_url(self):
        return self.redirected_url
    
    def parse_redirected_url(self, url):
        if 'error' in url:
            raise Exception("user denied access")
            
        
        spotify.set_redirected_url(url)
        query = urlparse(spotify.get_redirected_url())[5]
        parts = query.split('&')
    
        for i in range(len(parts)):
            parts[i] = parts[i].split('=')[1]
        return parts
    
    def set_access_token_and_expire(self, parts):
        self.access_token = parts[0]
        expires_in_seconds = int(parts[2])
        current_time = datetime.datetime.now()
        expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
        self.token_expires = expires
        self.access_token_did_expire = expires < current_time
    
    def get_access_token(self):
        return self.access_token
    
    ### different scopes must be space separated
    def set_scope(self, scope_string):
        self.scope = scope_string
        
    def get_scope(self):
        return self.scope

In [5]:
spotify = SpotifyAPI(client_id, client_secret);
spotify.set_scope("user-read-currently-playing")
url = spotify.perform_auth_grant_flow()

http://localhost:8888/callback#access_token=BQBPcqeUCQp4aX8LQpfvu2_5VOAzXd8p1ilRbw8pGcr0tm6WxSnZvsGcG5B_RJp5nRQUrVEDhKoH6bOvAMk9I1LFEwwT6tiVZxFoRamWNG63qBASthzgsxyIx5g8FIx2qttCGMTfPOdikbsR-4M2Sw0&token_type=Bearer&expires_in=3600


In [6]:
query = spotify.parse_redirected_url(url)
spotify.set_access_token_and_expire(query)

In [29]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {spotify.get_access_token()}"
}
endpoint = "https://api.spotify.com/v1/me/player/currently-playing?market=PL"
#data = urlencode({""})
lookup_url = f"{endpoint}"

r = requests.get(lookup_url, headers=headers)
print(r.json())

print(r.json()['item']['name'])

{'timestamp': 1616585980173, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z5u9yxbgdCldubbmMtLKN'}, 'href': 'https://api.spotify.com/v1/artists/4Z5u9yxbgdCldubbmMtLKN', 'type': 'artist', 'uri': 'spotify:artist:4Z5u9yxbgdCldubbmMtLKN'}, 'progress_ms': 307619, 'item': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z5u9yxbgdCldubbmMtLKN'}, 'href': 'https://api.spotify.com/v1/artists/4Z5u9yxbgdCldubbmMtLKN', 'id': '4Z5u9yxbgdCldubbmMtLKN', 'name': 'Exist', 'type': 'artist', 'uri': 'spotify:artist:4Z5u9yxbgdCldubbmMtLKN'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4Ygu5x67eMsDYIqfA4vflU'}, 'href': 'https://api.spotify.com/v1/albums/4Ygu5x67eMsDYIqfA4vflU', 'id': '4Ygu5x67eMsDYIqfA4vflU', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b2739d64531a6b3b1ea47cf93684', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e029d64531a6b3b1ea